In [1]:
"""
Lyapunov optimization for P2P net bit rate, 
one-way relay's error rate and two-way relay's outage probability.
Python ver.
"""
import math
import numpy as np
import gym
import time
import EH_P2P

np.random.seed(1)

Looptimes=70000
env=EH_P2P.EH_P2P()
env.Chanpower()
env.Solarread()
B=np.zeros(40)
modulation=0 #0：QPSK, 1:8PSK, 2:16QAM
protocol = 0  #0:DF, 1:AF

#mode: 0->P2P, 1->1-way relay, 2->2-way relay#
mode = 2



for snr in range(0,30,5):
    
    if mode == 0:
        s = env.reset_P2P(snr)
    elif mode == 1:
        s = env.reset_1_way(snr)
    else:
        s = env.reset_2_way(snr)
    V=10000
    Bt=0
    Btt=0
    RR=0
    R=B
    b=0
    RRMAX=0
    RMAX=0
    Sum=0
    aMAX=0
    threshold=300*40**(-snr/10)
    
    print("epoch, Virtual Queue B, battery, average net bit rate, action, snr")
    for t in range (Looptimes): 
        flag = 1
        #print(t)
        judge = 1
        end = 102
        if mode == 1:
            judge = env.judge_1_way()
        elif mode == 2:
            judge = env.judge_2_way(protocol)
            end = 4
            #threshold = 0
        if judge == 1:
            for ii in range (0,end,2):
                a=ii/100
                if mode == 0:
                    r,_ = env.search_P2P([a,modulation])
                    RR=2*V*r-2*Bt*a*b+2*b*a*b-(a*b)**2
                elif mode == 1:
                    r,_ = env.search_1_way(a)
                    RR=-2*V*r-2*Bt*a*b+2*b*a*b-(a*b)**2
                else:
                    a=a*100
                    a = 1
                    r,_ = env.search_2_way([a,protocol])
                    RR=-2*V*r-2*Bt*a*b-(a*b)**2
                    

            
                if flag==1:
                    RRMAX=RR
                    flag=0
                    a_decision=a
                if (RR>RRMAX):
                    a_decision=a
                    RRMAX=RR
        else: a_decision = 0
        #if judge == 1:print(a_decision)
        if mode == 0:
            s_,RMAX,INFO= env.step_P2P([a_decision,modulation])
        elif mode == 1:
            s_,RMAX,INFO= env.step_1_way(a_decision)
        else:
            s_,RMAX,INFO= env.step_2_way([a_decision,protocol])
        #print(RMAX, a_decision)
        if t % 10000==0:
            print(t,Bt,b,Sum/(t+1),a_decision, snr)
        b=s_[2]*300
        Btt=np.maximum(Bt-b+threshold,0)
        Bt=Btt
        Sum+=RMAX
        s=s_
    Results=Sum/Looptimes
    index=(snr+10)/2
    B[int(index)]=Results
    print(Results)

epoch, Virtual Queue B, battery, average net bit rate, action, snr
0 0 0 0.0 1 0
10000 26846.939822544187 147.14428296853336 0.7249275072492751 0 0
20000 151533.79956507622 21.764892067549987 0.72056397180141 0 0
30000 186969.06154275846 110.43435429074212 0.7214426185793806 1 0
40000 135559.31827960588 752.9179079114332 0.7201069973250669 1 0
50000 76743.45892716497 380.1947762369052 0.7219655606887863 0 0
60000 196026.8131232456 360.04935565127414 0.7215879735337745 1 0
0.7235857142857143
epoch, Virtual Queue B, battery, average net bit rate, action, snr
0 0 0 0.0 1 5
10000 135.1679324743541 98.12030905931961 0.22657734226577342 1 5
20000 0.0 317.56611558442904 0.20913954302284885 1 5
30000 0.0 208.84929271157682 0.21699276690776975 1 5
40000 0.0 54.59860556897094 0.2104947376315592 1 5
50000 0.0 77.30398056300648 0.2117957640847183 1 5
60000 0.0 146.45192307456185 0.21432976117064714 0 5
0.21365714285714285
epoch, Virtual Queue B, battery, average net bit rate, action, snr
0 0 0 0.0

In [2]:
np.savetxt("Lyap_P2P_1cm"+str(modulation)+".csv", B, delimiter = ',')

In [3]:
print(B)

[0.         0.         0.         0.         0.         0.72358571
 0.         0.21365714 0.         0.         0.0478     0.
 0.0109     0.         0.         0.00318571 0.         0.00095714
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
